# Power-Law

This program shows you how to calcualte the integrated photon and energy fluxes of the PL using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Power-Law :


Original Function, before integration: 
    
\begin{equation}
f\left(E\right) = N \times \left( \frac{E}{100} \right)^{\alpha} dE \ \ \ \ \ \ \ \         
\end{equation}

where $\alpha$ is the power-law index.

### My version of this equation:

Original Function, before integration: 
    
    a:     power-law index
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV


    f(E) = N * ((E/100.0)**a)

# Begin Program

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants and Parameters

In [2]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

pars     = [-1.492, 4.05928592E-02]  
# [plIndex, norm]

def get_parVals():
    pars     = [-1.492, 4.05928592E-02]  
    return pars


## Wolfram Alpha Integration of the Power-Law Function:

http://www.wolframalpha.com/input/?i=((x%2F100)%5Ea)+dx


When using Wolfram Alpha to integrate, be careful which letters you use for parameters.  Wolfram alpha has some letters set aside to mean something.  If they are used, you will not get the right answer. For example, E stands for exponential. Do NOT use E for energy.

N can be left out of integration. Simply multiply it back on at the end. The more parameters you have, the less likely Wolfram Alpha will calculate the function without a calculation time issue.

    a - power-law index
    N - normalization
    x - energy


In [3]:
def lpow(engs, flux, *params):
    plIdx = float(params[0])

    for i in range(len(engs)-1):
        lowIntegral     = (100.**(-plIdx)) * ((engs[i]**(plIdx+1.)) /(plIdx + 1.))
        highIntegral    = (100.**(-plIdx)) * ((engs[i+1]**(plIdx+1.))/(plIdx + 1.))
        val     = (highIntegral - lowIntegral)
        flux[i] = val

In [4]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
lpow(engs, flux, *pars)
norm   = pars[-1]
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN ESBPL FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))


Photon Flux:  24.758399542 	 photons s^-1 cm^-2
Energy Flux:  1.287664172e-05 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.

In [5]:
def pl(energy):
    L,N    = get_parVals()
    eng    = energy
    return N * ((eng/100.0)**L)

def epl(energy):
    eng    = energy
    return eng * pl(eng)

In [6]:
Flux_Ph = integrate.quad(pl, emin, emax, limit=10000)[0]
Flux_En = integrate.quad(epl, emin, emax, limit=10000)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  24.758399542 	 photons s^-1 cm^-2
Energy Flux:  1.288554146e-05 	 ergs s^-1 cm^-2

